# Glow-TTS Training 02

## Google Drive

In [2]:
# mount google drive for data storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Coqui TTS

In [2]:
# upgrade pip, install tts, and upgrade tensorflow
! pip install -U pip
! pip install TTS
! pip install -U tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 40.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 73.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 35.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 79.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successful

## Imports

In [2]:
# import the necessary libraries
import os
import glob

from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig

## Load Dataset

In [3]:
# set output path under google drive directory
output_path = "/content/drive/MyDrive/ljspeech-002/tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

# set the dataset configuration
# ljspeech dataset was chosen, but only one speaker (002) will be used
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="/content/drive/MyDrive/ljspeech-002/metadata.csv", path="/content/drive/MyDrive/ljspeech-002"
)

## Config

In [4]:
# model architecture and configuration
# 250 epochs will be trained with 32 training batch size
# checkpoints every 200 steps, meaning around every 19 epochs,
# since 337 training samples / 32 batch size = 10.5 steps per epoch
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=250,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=200,
)

## Audio Processor

In [5]:
# audio processor for feature extraction and audio input/output
ap = AudioProcessor.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


## Tokenizer

In [6]:
# tokenizer for converting text to sequences of token ids
tokenizer, config = TTSTokenizer.init_from_config(config)

## Load Data Samples

In [7]:
# load the training and evaluation samples from the dataset
# each sample is a list of [text, audio_file_path, speaker_name]
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 337 files in /content/drive/MyDrive/ljspeech-002


## Model Initialization

In [8]:
# model initialization with configuration, audio processor, tokenizer, and no speaker manager
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

# trainer generic api for training the model
# here, we also restore the model from the last checkpoint and continue training,
# because it was trained for 2400+ steps before the runtime was disconnected
trainer = Trainer(
    TrainerArgs(restore_path=output_path+'/run-June-19-2024_05+47PM-0000000/checkpoint_2400.pth'),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-21-2024_01+45PM-0000000
 > Restoring from checkpoint_2400.pth ...
 > Restoring Model...
 > Restoring Optimizer...
 > Model restored from step 2400
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(

 > Model has 28610257 parameters


## Training

In [ ]:
# start the training process
trainer.fit()


 > EPOCH: 0/250
 --> /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-21-2024_12+06PM-0000000




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 334


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > TRAINING (2024-06-21 12:12:32) 


 | > Preprocessing samples
 | > Max text length: 179
 | > Min text length: 32
 | > Avg text length: 107.95808383233533
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 33971.0
 | > Avg audio length: 146823.3113772455
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

 > EVALUATION 



...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.650470495223999 (+0.06144547462463379)
     | > avg_loss: 0.1469622552394867 (+0.0026790201663970947)
     | > avg_log_mle: -0.13293921947479248 (-0.0007184743881225586)
     | > avg_loss_dur: 0.2799014747142792 (+0.0033974945545196533)


 > EPOCH: 67/250
 --> /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-21-2024_12+06PM-0000000

 > TRAINING (2024-06-21 12:53:43) 

 > EVALUATION 



 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.6241376399993896 (-0.026332855224609375)
     | > avg_loss: 0.13371491432189941 (-0.01324734091758728)
     | > avg_log_mle: -0.13323307037353516 (-0.0002938508987426758)
     | > avg_loss_dur: 0.26694798469543457 (-0.012953490018844604)

 > BEST MODEL : /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-21-2024_12+06PM-0000000/best_model_3149.pth

 > EPOCH: 68/250
 --> /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-21-2024_12+06PM-0000000

 > TRAINING (2024-06-21 12:54:21) 

   --> TIME: 2024-06-21 12:54:30 -- STEP: 1/11 -- GLOBAL_STEP: 3150
     | > loss: 0.09258395433425903  (0.09258395433425903)
     | > log_mle: -0.17499899864196777  (-0.17499899864196777)
     | > loss_dur: 0.2675829529762268  (0.2675829529762268)
     | > amp_scaler: 65536.0  (65536.0)
     | > grad_norm: tensor(1.4471, device='cuda:0')  (tensor(1.4471, device='cuda:0'))
     | > current_lr: 1.7e-05 
     | > step_time: 2.234  (2.23395061492

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.9279077053070068 (+0.3037700653076172)
     | > avg_loss: 0.13156896829605103 (-0.0021459460258483887)
     | > avg_log_mle: -0.13351547718048096 (-0.0002824068069458008)
     | > avg_loss_dur: 0.265084445476532 (-0.0018635392189025879)

